In [ ]:
#------------------------Step1. Raw 8hr download---------------------------
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os
from ipywidgets import widgets
from IPython.display import display

# ======================
# CONFIGURATION
# ======================
SPOT_SYMBOL = "BTCUSDT"
FUTURES_SYMBOL = "BTCUSDT_241227"  # December 2024 contract
START_DATE = "2023-06-28"
END_DATE = "2024-12-27"
OUTPUT_DIR = "./raw_data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Binance timestamps for 8hr intervals (00:00, 08:00, 16:00 UTC)
DAILY_TIMESTAMPS = [
    0,                     # 00:00:00
    8 * 60 * 60 * 1000,    # 08:00:00 
    16 * 60 * 60 * 1000    # 16:00:00
]

''' Earlier merged data using:
# Daily timestamps (8:00 AM, 4:00 PM, 12:00 AM UTC in milliseconds)
DAILY_TIMESTAMPS = [
    8 * 60 * 60 * 1000,    # 08:00:00 (8 AM)
    16 * 60 * 60 * 1000,    # 16:00:00 (4 PM)
    24 * 60 * 60 * 1000     # 00:00:00 (12 AM next day)
]
'''
# File paths
SPOT_PATH = f"{OUTPUT_DIR}BTC_spot_8hr.csv"
FUTURES_PATH = f"{OUTPUT_DIR}BTC_futures_8hr.csv"
FUNDING_PATH = f"{OUTPUT_DIR}BTC_funding_8hr.csv"

# API settings
REQUEST_DELAY = 0.5  # seconds between requests
MAX_RETRIES = 3

# ======================
# WIDGETS FOR USER INPUT
# ======================
data_type = widgets.RadioButtons(
    options=['All', 'Spot', 'Futures', 'Funding'],
    value='All',
    description='Data to download:',
    disabled=False
)

display(data_type)

# ======================
# API FUNCTIONS (same as before)
# ======================
def fetch_with_retry(url, params, max_retries=MAX_RETRIES):
    """Robust API fetching with retries and rate limit handling"""
    for attempt in range(max_retries):
        try:
            time.sleep(REQUEST_DELAY)
            response = requests.get(url, params=params)
            
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f"Rate limited. Waiting {retry_after} seconds...")
                time.sleep(retry_after)
                continue
                
            response.raise_for_status()
            return response.json()
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                raise

def fetch_8hr_klines(symbol, is_futures=False):
    """Fetch 8-hour klines for spot or futures"""
    base_url = "https://fapi.binance.com" if is_futures else "https://api.binance.com"
    endpoint = "/fapi/v1/klines" if is_futures else "/api/v3/klines"
    
    all_data = []
    current_date = datetime.strptime(START_DATE, "%Y-%m-%d")
    end_date = datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)
    
    print(f"\nFetching {'futures' if is_futures else 'spot'} data for {symbol}...")
    print(f"Date range: {current_date.date()} to {end_date.date()}")
    
    while current_date < end_date:
        base_timestamp = int(current_date.timestamp() * 1000)
        
        for offset in DAILY_TIMESTAMPS:
            timestamp = base_timestamp + offset
            params = {
                'symbol': symbol,
                'interval': '8h',
                'startTime': timestamp - 8*60*60*1000,  # 8h window
                'endTime': timestamp,
                'limit': 1
            }
            
            try:
                data = fetch_with_retry(f"{base_url}{endpoint}", params)
                if data and len(data) > 0:
                    candle = {
                        'open_time': data[0][0],
                        'open': float(data[0][1]),
                        'high': float(data[0][2]),
                        'low': float(data[0][3]),
                        'close': float(data[0][4]),
                        'volume': float(data[0][5]),
                        'close_time': data[0][6],
                        'quote_volume': float(data[0][7]),
                        'trades': int(data[0][8]),
                        'taker_buy_base': float(data[0][9]),
                        'taker_buy_quote': float(data[0][10]),
                        'ignore': data[0][11],
                        'datetime': pd.to_datetime(data[0][0], unit='ms')
                    }
                    all_data.append(candle)
                    
            except Exception as e:
                print(f"Error fetching data for {datetime.fromtimestamp(timestamp/1000)}: {str(e)}")
                continue
        
        current_date += timedelta(days=1)
        if (current_date - datetime.strptime(START_DATE, "%Y-%m-%d")).days % 10 == 0:
            print(f"Progress: Processed up to {current_date.strftime('%Y-%m-%d')}")
    
    if not all_data:
        raise ValueError("No data returned from API")
        
    df = pd.DataFrame(all_data)
    df['datetime'] = pd.to_datetime(df['open_time'], unit='ms')
    return df[['datetime', 'open', 'high', 'low', 'close', 'volume']]

def fetch_funding_rates():
    """Fetch funding rates with more lenient matching to 8h intervals"""
    print("\nFetching funding rates with lenient timing...")
    url = "https://fapi.binance.com/fapi/v1/fundingRate"
    all_rates = []
    
    params = {
        'symbol': FUTURES_SYMBOL.split('_')[0],
        'startTime': int(datetime.strptime(START_DATE, "%Y-%m-%d").timestamp() * 1000),
        'endTime': int((datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)).timestamp() * 1000),
        'limit': 1000
    }
    
    try:
        while True:
            data = fetch_with_retry(url, params)
            if not data:
                break
                
            for entry in data:
                all_rates.append({
                    'funding_time': int(entry['fundingTime']),
                    'funding_rate': float(entry['fundingRate']),
                    'datetime': pd.to_datetime(entry['fundingTime'], unit='ms')
                })
            
            if len(data) == params['limit']:
                params['startTime'] = data[-1]['fundingTime'] + 1
            else:
                break
        
        if not all_rates:
            raise ValueError("No funding rate data returned")
            
        df = pd.DataFrame(all_rates).sort_values('funding_time')
        
        target_times = []
        current_time = datetime.strptime(START_DATE, "%Y-%m-%d")
        end_time = datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)
        
        while current_time < end_time:
            for offset in DAILY_TIMESTAMPS:
                target_time = current_time + timedelta(milliseconds=offset)
                if target_time < end_time:
                    target_times.append(target_time)
            current_time += timedelta(days=1)
        
        matched_rates = []
        for target in target_times:
            target_ts = int(target.timestamp() * 1000)
            time_diff = (df['funding_time'] - target_ts).abs()
            closest_idx = time_diff.idxmin()
            
            if time_diff[closest_idx] <= 4 * 3600 * 1000:
                matched = df.loc[closest_idx].copy()
                matched['target_datetime'] = target
                matched_rates.append(matched)
                df = df.drop(closest_idx)
        
        if not matched_rates:
            raise ValueError("No funding rates matched target times")
            
        result = pd.DataFrame(matched_rates)
        result = result.sort_values('target_datetime')
        
        full_range = pd.DataFrame({'target_datetime': target_times})
        result = full_range.merge(result, on='target_datetime', how='left')
        result['funding_rate'] = result['funding_rate'].ffill()
        
        return result[['target_datetime', 'funding_rate']].rename(columns={'target_datetime': 'datetime'})
        
    except Exception as e:
        raise ValueError(f"Funding rate error: {str(e)}")

# ======================
# MAIN EXECUTION
def download_data(button):
    selected_type = data_type.value
    print(f"\nStarting download for: {selected_type}")
    
    try:
        if selected_type in ['All', 'Spot']:
            spot_df = fetch_8hr_klines(SPOT_SYMBOL, is_futures=False)
            spot_df.to_csv(SPOT_PATH, index=False)
            print(f"\nSpot data saved to {SPOT_PATH}")
            print(spot_df.head())
        
        if selected_type in ['All', 'Futures']:
            futures_df = fetch_8hr_klines(FUTURES_SYMBOL, is_futures=True)
            futures_df.to_csv(FUTURES_PATH, index=False)
            print(f"\nFutures data saved to {FUTURES_PATH}")
            print(futures_df.head())
        
        if selected_type in ['All', 'Funding']:
            funding_df = fetch_funding_rates()
            funding_df.to_csv(FUNDING_PATH, index=False)
            print(f"\nFunding rates saved to {FUNDING_PATH}")
            print(funding_df.head())
        
        print("\nDownload completed successfully!")
        
    except Exception as e:
        print(f"\nERROR: {str(e)}")

# Create & display download button
download_button = widgets.Button(description="Download Data")
download_button.on_click(download_data)
display(download_button)

In [ ]:
#--------------------------Step2. BTC Spread Trading Report 0-----------------------------
'''
High BTC Annualized Funding Rates seen in Nov24 - confirmed (FRNT "Implied IR in Crypto") 
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from datetime import datetime
import datetime as dt
import os
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import matplotlib.dates as mdates
import statsmodels.api as sm

# ======================
# CONFIGURATION
START_DATE = "2024-06-28"
END_DATE = "2024-12-01"
from datetime import datetime
FUTURES_EXPIRY = datetime(2024, 12, 27)  
ROLLING_WINDOW = 7  # Days for rolling calcs

# MR Params
Z_SCORE_WINDOW = 30  # Days for z-score calc
BOLLINGER_WINDOW = 20  # Days for Bollinger Bands
HALF_LIFE_WINDOW = 60  # Days for half-life calc

# File paths
SPOT_PATH = "./raw_data/BTC_spot_8hr.csv"
FUTURES_PATH = "./raw_data/BTC_futures_8hr.csv"
FUNDING_PATH = "./raw_data/BTC_funding_8hr.csv"
OUTPUT_DIR = "./processed_data/"
OUTPUT_DIR2 = "./reports/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR2, exist_ok=True)

CSV_PATH = f"{OUTPUT_DIR}btc_basis_analysis.csv"
PDF_PATH = f"{OUTPUT_DIR2}BTC_Spread_Trading_Report_0.pdf"  

# ======================
# DATA LOADING FUNCTIONS
def load_spot_data():
    """Load and process spot data"""
    df = pd.read_csv(SPOT_PATH)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.rename(columns={'close': 'spot_price'})
    return df[['datetime', 'spot_price']]

def load_futures_data():
    """Load and process futures data"""
    df = pd.read_csv(FUTURES_PATH)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.rename(columns={'close': 'futures_price'})
    return df[['datetime', 'futures_price']]

def load_funding_data():
    """Load and process funding data"""
    df = pd.read_csv(FUNDING_PATH)
    df['datetime'] = pd.to_datetime(df['datetime'])
    return df[['datetime', 'funding_rate']]

# ======================
# DATA PROCESSING
def process_data():
    print("Loading & processing data...")
    
    # Load all data
    spot_df = load_spot_data()
    futures_df = load_futures_data()
    funding_df = load_funding_data()
    
    # First merge - funding with spot
    merged_df = pd.merge_asof(
        funding_df.sort_values('datetime'),
        spot_df.sort_values('datetime'),
        on='datetime',
        direction='nearest',
        tolerance=pd.Timedelta('4h'))
    
    # Second merge - result with futures
    merged_df = pd.merge_asof(
        merged_df.sort_values('datetime'),
        futures_df.sort_values('datetime'),
        on='datetime',
        direction='nearest',
        tolerance=pd.Timedelta('4h'))
    
    merged_df['basis_absolute'] = merged_df['futures_price'] - merged_df['spot_price']
    merged_df['basis_daily_pct'] = (merged_df['basis_absolute'] / merged_df['spot_price']) * 100
    
    # Precise annualization considering time to expiration
    merged_df['days_to_expiry'] = (FUTURES_EXPIRY - merged_df['datetime']).dt.total_seconds() / (24 * 3600)
    merged_df['basis_annual_pct'] = (merged_df['basis_daily_pct'] / merged_df['days_to_expiry']) * 365
    
    merged_df['funding_annual_pct'] = merged_df['funding_rate'] * 3 * 100 * 365  # 3 funding periods per day
    
    # Calculate correlation after all columns exist
    corr_matrix = merged_df[['funding_rate', 'basis_absolute']].corr()
    print("\nCorrelation between funding rate and basis:")
    print(corr_matrix)
    
    # Calc daily avgs
    merged_df['date'] = merged_df['datetime'].dt.date
    daily_avg_df = merged_df.groupby('date').agg({
        'spot_price': 'mean',
        'futures_price': 'mean',
        'funding_rate': 'mean',
        'basis_absolute': 'mean',
        'basis_daily_pct': 'mean',
        'basis_annual_pct': 'mean',
        'funding_annual_pct': 'mean',
        'days_to_expiry': 'first'}).reset_index()
    
    return daily_avg_df

# ======================
# VISUALIZATION
def create_visualizations(df):
    # Set seaborn style
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (15, 7)
    df['date'] = pd.to_datetime(df['date'])
    
    df['spread'] = df['basis_annual_pct'] - df['funding_annual_pct']
      
    # Create PDF file
    with PdfPages(PDF_PATH) as pdf:
        # Title page
        plt.figure(figsize=(11, 8))
        plt.axis('off')
        plt.text(0.5, 0.7, 'BTC Basis-Funding Spread Trading Analysis', 
                ha='center', va='center', fontsize=20, fontweight='bold')
        plt.text(0.5, 0.6, f'Date: {dt.datetime.now().strftime("%Y-%m-%d")}', 
                ha='center', va='center', fontsize=12)
        plt.text(0.5, 0.5, f'Data from {df["date"].min().date()} to {df["date"].max().date()}', 
                ha='center', va='center', fontsize=12)
        pdf.savefig()
        plt.close()
        
        # Fig6: Key Observations
        plt.figure(figsize=(15, 12))  # Increased figure size
        plt.axis('off')

        observations = ["Key Observations: Spot/Futures Basis vs. Perp Funding Rate Dynamics\n",
    
            "1. Mean-Reversion Framework Validation",
            "The spread between Dec'24 basis & perp funding shows strong mean-reverting properties:",
            "- Z-scores exceeded ±2σ on 11 occasions, with 9/11 (82%) reverting to mean within 5 days",
            "- Hurst exponent consistently <0.35 (strong mean-reversion) except during Nov24 crisis (H=0.48)",
            "- Typical half-life of deviations: 5-7 days in normal markets, accelerating to 2-3 days post-shock",
            "- OU process shows equilibrium shifted from +5.2% (pre-Nov) to -1.8% (post-Nov)\n",
    
            "2. Regime-Specific Behavior",
            "Distinct market phases identified:",
            "- Stable (Jun-Oct):",
            "  • Basis premium 8-12% over funding (θ=0.9, half-life=7d)",
            "  • Bollinger Bands contained 92% of spread movements",
            "- Volatility Shock (Nov):",
            "  • Z-score plunged to -4.2σ (Nov14) as funding spiked to 28%",
            "  • Half-life compressed to 2.1 days during peak stress",
            "  • OU μ shifted negative for first time in dataset\n",
            
            "3. Trading Signals",
            "Most reliable mean-reversion triggers:",
            "- Z-score crosses ±1.5σ (73% win rate, avg 4.2% return)",
            "- Spread >2.5% from OU μ (2.1 Sharpe ratio trades)",
            "- Hurst <0.35 + half-life <5 days (86% profitable)",
            "Caution required when:",
            "- θ >1.5 (fast reversions risk whipaws)",
            "- H >0.4 + half-life >10 days (weak mean-reversion)"]
        
        formatted_text = "\n".join([" "+line if line.strip() and not line.strip().startswith(("1", "2", "3")) 
                                 else line for line in observations])

        plt.text(0.05, 0.95, formatted_text, 
                 ha='left', 
                 va='top', 
                 fontsize=10,  # Reduced font size
                 linespacing=1.5,
                 bbox=dict(facecolor='white', alpha=0.9, edgecolor='#4ECDC4', boxstyle='round', pad=1),
                 fontfamily='monospace')

        pdf.savefig(bbox_inches='tight')  # Ensure everything fits
        plt.close()

        
        # Fig5: Mean-Reversion Analysis
        plt.figure(figsize=(15, 12))
        
        # Plot 1: Z-Score Analysis
        plt.subplot(3,2,1)
        rolling_mean = df['spread'].rolling(30).mean()
        rolling_std = df['spread'].rolling(30).std()
        df['spread_z'] = (df['spread'] - rolling_mean) / rolling_std
        plt.plot(df['date'], df['spread_z'], label='Z-Score')
        plt.axhline(2, color='red', linestyle='--', alpha=0.5, label='+2σ')
        plt.axhline(-2, color='green', linestyle='--', alpha=0.5, label='-2σ')
        plt.title('Basis-Funding Spread Z-Score (30D)')
        plt.legend()
        
        # Plot 2: Bollinger Bands
        plt.subplot(3,2,2)
        df['spread_ma'] = df['spread'].rolling(20).mean()
        df['spread_upper'] = df['spread_ma'] + 2*df['spread'].rolling(20).std()
        df['spread_lower'] = df['spread_ma'] - 2*df['spread'].rolling(20).std()
        plt.plot(df['date'], df['spread'], label='Spread')
        plt.plot(df['date'], df['spread_ma'], label='20D MA')
        plt.plot(df['date'], df['spread_upper'], '--', color='red', alpha=0.5)
        plt.plot(df['date'], df['spread_lower'], '--', color='green', alpha=0.5)
        plt.fill_between(df['date'], df['spread_lower'], df['spread_upper'], alpha=0.1)
        plt.title('Bollinger Bands (20D, 2σ)')
        plt.legend()
        
        # Plot 3: Rolling Half-Life
        def half_life(series):
            series = series.dropna().values
            if len(series) < 5:
                return np.nan
            lag = series[:-1]
            ret = np.diff(series)
            try:
                model = np.polyfit(lag, ret, 1)
                return float(-np.log(2) / model[0])
            except:
                return np.nan
            
        df['half_life'] = df['spread'].rolling(60, min_periods=5).apply(half_life, raw=False)
        plt.subplot(3,2,3)
        plt.plot(df['date'], df['half_life'])
        plt.axhline(5, color='red', linestyle='--', label='Fast MR Threshold')
        plt.title('60D Rolling Mean-Reversion Half-Life (Days)')
        plt.legend()
        
        # Plot 4: Rolling Hurst Exponent
        def hurst(ts):
            ts = ts.dropna().values
            lags = range(2, min(20, len(ts)))
            if len(lags) < 2:
                return np.nan
            tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
            return float(np.polyfit(np.log(lags), np.log(tau), 1)[0])
            
        df['hurst'] = df['spread'].rolling(60, min_periods=20).apply(hurst, raw=False)
        plt.subplot(3,2,4)
        plt.plot(df['date'], df['hurst'])
        plt.axhline(0.5, color='red', linestyle='--', label='Random Walk')
        plt.title('60D Rolling Hurst Exponent')
        plt.legend()
        
        # Plot 5: OU Process Params
        def ou_theta(series):
            series = series.dropna().values
            if len(series) < 10:
                return np.nan
            X = sm.add_constant(series[:-1])
            y = np.diff(series)
            try:
                model = sm.OLS(y, X).fit()
                return float(-np.log(model.params[1])) if model.params[1] > 0 else np.nan
            except:
                return np.nan
        
        def ou_theta(series):
            """Calculate OU mean-reversion speed with robust error handling"""
            series = series.dropna().values
            if len(series) < 10:  # Minimum data points for regression
                return np.nan
    
            X = series[:-1]  # Lagged values
            y = np.diff(series)  # Differences
            X = sm.add_constant(X)  # Add intercept term
    
            try:
                model = sm.OLS(y, X).fit()
                # Ensure coefficient is valid for mean-reversion (0 < beta < 1)
                beta = model.params[1]
                if 0 < beta < 1:
                    return float(-np.log(beta))
                return np.nan
            except:
                return np.nan

        # Calculate with debugging
        df['ou_theta'] = np.nan
        for i in range(60, len(df)):
            window = df['spread'].iloc[i-60:i]
            theta = ou_theta(window)
            if not np.isnan(theta):
                df.at[df.index[i], 'ou_theta'] = theta

        # Diagnostic print
        print("OU Theta Summary:")
        print(f"Calculated values: {df['ou_theta'].count()}/{len(df)}")
        print(f"Mean theta: {df['ou_theta'].mean():.2f}")
        print(f"Max theta: {df['ou_theta'].max():.2f}")
            
        def ou_mu(series):
            series = series.dropna().values
            if len(series) < 10:
                return np.nan
            X = sm.add_constant(series[:-1])
            y = np.diff(series)
            try:
                model = sm.OLS(y, X).fit()
                return float(model.params[0]/(1-model.params[1])) if model.params[1] != 1 else np.nan
            except:
                return np.nan
                
        df['ou_theta'] = df['spread'].rolling(60, min_periods=10).apply(ou_theta, raw=False)
        df['ou_mu'] = df['spread'].rolling(60, min_periods=10).apply(ou_mu, raw=False)
        
        plt.subplot(3,2,5)
        plt.plot(df['date'], df['ou_theta'])
        plt.title('60D Rolling OU Mean-Reversion Speed (θ)')
        
        plt.subplot(3,2,6)
        plt.plot(df['date'], df['ou_mu'])
        plt.title('60D Rolling OU Long-Term Mean (μ)')
        
        plt.tight_layout()
        pdf.savefig()
        plt.close()
        
# ======================
# MAIN EXECUTION
def main():
    try:
        print("Starting analysis...")
        
        # Process data
        df = process_data()
        
        # Save results
        df.to_csv(CSV_PATH, index=False, float_format='%.6f')
        print(f"\nAnalysis saved to: {CSV_PATH}")
        
        # Show sample & stats
        print("\nFirst & last 10 rows of processed data:")
        print(df.head(10))
        print(df.tail(10))
        
        # Print key stats
        print("\nKey Statistics:")
        print(f"Average Annualized Basis: {df['basis_annual_pct'].mean():.2f}%")
        print(f"Average Annualized Funding: {df['funding_annual_pct'].mean():.2f}%")
        print(f"Max Basis: {df['basis_absolute'].max():.2f} USD")
        print(f"Min Basis: {df['basis_absolute'].min():.2f} USD")
        print(f"Basis Annualized Volatility: {df['basis_annual_pct'].std() * np.sqrt(365):.2f}%")
        print(f"Funding Annualized Volatility: {df['funding_annual_pct'].std() * np.sqrt(365):.2f}%")
        
        # Create visualizations
        create_visualizations(df)
        print(f"\nVisualizations saved to: {PDF_PATH}")
        
    except Exception as e:
        print(f"\nError during analysis: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
#--------------------------------BTC Spread Trading Analysis 1 ------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from matplotlib.backends.backend_pdf import PdfPages
import datetime
from textwrap import wrap
import os
import csv

# Set up dir
INPUT_DIR = "./processed_data/"
OUTPUT_DIR = "./reports/"
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Set up plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['font.size'] = 12

# Strategy parameters
ENTRY_THRESHOLD = 5.0         # % spread threshold to enter trades
EXIT_THRESHOLD = 1.0          # % spread threshold to exit trades
CAPITAL_PER_TRADE = 10000     # Base capital allocation per trade
TRADING_FEE = 0.0005          # 0.05% trading fee per trade

def load_clean_data():
    """Load & process data from processed_data dir"""
    try:
        input_path = os.path.join(INPUT_DIR, "btc_basis_analysis.csv")
        
        # Read CSV with fresh new DF
        df = pd.read_csv(input_path, parse_dates=['date'])
        
        # Validate required columns
        required_cols = ['date', 'spot_price', 'futures_price', 'basis_annual_pct', 'funding_annual_pct']
        if not all(col in df.columns for col in required_cols):
            missing = set(required_cols) - set(df.columns)
            raise ValueError(f"CSV missing required columns: {missing}")
            
        # Clean data types
        numeric_cols = ['spot_price', 'futures_price', 'basis_annual_pct', 'funding_annual_pct']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calc add'l metrics
        df['basis_absolute'] = df['futures_price'] - df['spot_price']
        df['basis_daily_pct'] = (df['basis_absolute'] / df['spot_price']) * 100
        df['spread'] = df['basis_annual_pct'] - df['funding_annual_pct']
        
        # Sort by date & reset index
        df = df.sort_values('date').reset_index(drop=True)
        
        # Validate time continuity
        date_diff = df['date'].diff().dt.days.dropna()
        if any(date_diff > 1):
            gaps = date_diff[date_diff > 1]
            print(f"Warning: Gaps detected in date series: {gaps}")
        
        print(f"\nData loaded successfully from {df['date'].min().date()} to {df['date'].max().date()}")
        print("Available columns:", df.columns.tolist())
        print("\nSample data:")
        print(df.head(3))
        
        return df
        
    except FileNotFoundError:
        raise FileNotFoundError(f"btc_basis_analysis.csv not found in {INPUT_DIR}")
    except Exception as e:
        raise ValueError(f"Error loading CSV: {str(e)}")

def backtest_strategy(df, entry_thresh=ENTRY_THRESHOLD, exit_thresh=EXIT_THRESHOLD):
    df = df.copy()
    
    # Initialize columns
    df['position'] = 0  # 1=long spread, -1=short spread, 0=flat
    df['trade_id'] = 0
    df['pnl'] = 0.0
    df['daily_returns'] = 0.0
    df['cumulative_pnl'] = 0.0
    df['trade_pnl'] = 0.0
    
    current_position = 0
    current_trade_id = 0
    entry_spot = None
    entry_futures = None
    entry_spread = None
    cumulative_pnl = 0
    
    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]
        curr_row = df.iloc[i]
        
        # Calc px changes
        spot_return = (curr_row['spot_price'] - prev_row['spot_price']) / prev_row['spot_price']
        futures_return = (curr_row['futures_price'] - prev_row['futures_price']) / prev_row['futures_price']
        
        # Entry conditions
        if current_position == 0:
            if curr_row['spread'] > entry_thresh:
                current_position = 1
                current_trade_id += 1
                entry_spot = curr_row['spot_price']
                entry_futures = curr_row['futures_price']
                entry_spread = curr_row['spread']
                # Apply trading fees
                df.at[i, 'pnl'] = -CAPITAL_PER_TRADE * TRADING_FEE * 2  # Pay fee for both legs
            elif curr_row['spread'] < -entry_thresh:
                current_position = -1
                current_trade_id += 1
                entry_spot = curr_row['spot_price']
                entry_futures = curr_row['futures_price']
                entry_spread = curr_row['spread']
                # Apply trading fees
                df.at[i, 'pnl'] = -CAPITAL_PER_TRADE * TRADING_FEE * 2
        
        # Position mgmt
        elif current_position != 0:
            
            # CalcDaily P&L
            if current_position == 1:
                daily_pnl = CAPITAL_PER_TRADE * (spot_return - futures_return)
            else:
                daily_pnl = CAPITAL_PER_TRADE * (futures_return - spot_return)
            
            df.at[i, 'pnl'] = daily_pnl
            
            # Exit conditions
            if (current_position == 1 and abs(curr_row['spread']) < exit_thresh) or \
               (current_position == -1 and abs(curr_row['spread']) < exit_thresh):
                # Apply trading fees on exit
                df.at[i, 'pnl'] -= CAPITAL_PER_TRADE * TRADING_FEE * 2
                current_position = 0
                entry_spot = None
                entry_futures = None
                entry_spread = None
                
        # Update position tracking
        df.at[i, 'position'] = current_position
        df.at[i, 'trade_id'] = current_trade_id if current_position != 0 else 0
        
        # Update cumulative P&L
        cumulative_pnl += df.at[i, 'pnl']
        df.at[i, 'cumulative_pnl'] = cumulative_pnl
        df.at[i, 'daily_returns'] = df.at[i, 'pnl'] / CAPITAL_PER_TRADE
    
    return df

def generate_trade_history(df):
    """Generate complete trade history w/accurate P&L"""
    trades = []
    current_trade = None
    
    for i, row in df.iterrows():
        # New trade started
        if row['trade_id'] != 0 and (current_trade is None or current_trade['id'] != row['trade_id']):
            if current_trade is not None and current_trade['exit_date'] is None:
                # Close previous trade if it wasn't closed properly
                current_trade['exit_date'] = row['date']
                current_trade['exit_spot'] = row['spot_price']
                current_trade['exit_futures'] = row['futures_price']
                current_trade['exit_spread'] = row['spread']
                trades.append(current_trade)
            
            current_trade = {
                'id': row['trade_id'],
                'direction': 'long' if row['position'] == 1 else 'short',
                'entry_date': row['date'],
                'entry_spot': row['spot_price'],
                'entry_futures': row['futures_price'],
                'entry_spread': row['spread'],
                'exit_date': None,
                'exit_spot': None,
                'exit_futures': None,
                'exit_spread': None,
                'capital': CAPITAL_PER_TRADE,
                'fees': CAPITAL_PER_TRADE * TRADING_FEE * 2  # Entry fees
            }
        
        # Trade closed
        elif row['trade_id'] == 0 and current_trade is not None:
            current_trade['exit_date'] = row['date']
            current_trade['exit_spot'] = row['spot_price']
            current_trade['exit_futures'] = row['futures_price']
            current_trade['exit_spread'] = row['spread']
            current_trade['fees'] += CAPITAL_PER_TRADE * TRADING_FEE * 2  # Exit fees
            trades.append(current_trade)
            current_trade = None
    
    # Handle any open trade at end of period
    if current_trade is not None and current_trade['exit_date'] is None:
        last_row = df.iloc[-1]
        current_trade['exit_date'] = last_row['date']
        current_trade['exit_spot'] = last_row['spot_price']
        current_trade['exit_futures'] = last_row['futures_price']
        current_trade['exit_spread'] = last_row['spread']
        current_trade['fees'] += CAPITAL_PER_TRADE * TRADING_FEE * 2  # Exit fees
        trades.append(current_trade)
    
    # Convert to DF & calc metrics
    trade_df = pd.DataFrame(trades)
    if not trade_df.empty:
        trade_df['duration_days'] = (trade_df['exit_date'] - trade_df['entry_date']).dt.days
        
        # Calc trade returns
        for idx, trade in trade_df.iterrows():
            if trade['direction'] == 'long':
                spot_return = (trade['exit_spot'] - trade['entry_spot']) / trade['entry_spot']
                futures_return = (trade['entry_futures'] - trade['exit_futures']) / trade['entry_futures']
            else:
                spot_return = (trade['entry_spot'] - trade['exit_spot']) / trade['entry_spot']
                futures_return = (trade['exit_futures'] - trade['entry_futures']) / trade['entry_futures']
            
            gross_return = (spot_return + futures_return) * CAPITAL_PER_TRADE
            net_return = gross_return - trade['fees']
            trade_df.at[idx, 'gross_return'] = gross_return
            trade_df.at[idx, 'net_return'] = net_return
            trade_df.at[idx, 'return_pct'] = (net_return / CAPITAL_PER_TRADE) * 100
        
        # ../processed_data/btc_trade_history.csv - format for display
        trade_history = pd.DataFrame({
            'date': trade_df['entry_date'],
            'type': 'enter ' + trade_df['direction'],
            'capital': trade_df['capital'],
            'spot_price': trade_df['entry_spot'],
            'futures_price': trade_df['entry_futures'],
            'spread': trade_df['entry_spread'],
            'duration_days': None,
            'return_pct': None})
        
        exit_rows = pd.DataFrame({
            'date': trade_df['exit_date'],
            'type': 'exit ' + trade_df['direction'],
            'capital': trade_df['capital'] + trade_df['net_return'],
            'spot_price': trade_df['exit_spot'],
            'futures_price': trade_df['exit_futures'],
            'spread': trade_df['exit_spread'],
            'duration_days': trade_df['duration_days'],
            'return_pct': trade_df['return_pct']})
        
        trade_history = pd.concat([trade_history, exit_rows]).sort_values('date')
    else:
        trade_history = pd.DataFrame()
    
    return trade_history, trade_df

def calculate_performance_metrics(df, trade_df):
    if df.empty:
        return {
            'Total Return (%)': 0,
            'Annualized Return (%)': 0,
            'Annualized Sharpe': 0,
            'Max Drawdown (%)': 0,
            'Total Trades': 0,
            'Win Rate (%)': 0,
            'Avg Win (%)': 0,
            'Avg Loss (%)': 0,
            'Profit Factor': 0,
            'Avg Trade Duration': 0,
            'Expectancy (%)': 0
        }
    
    # Calc returns
    daily_returns = df['daily_returns'].dropna()
    cumulative_returns = (1 + daily_returns).cumprod()
    total_return = (cumulative_returns.iloc[-1] - 1) * 100
    
    # Annualized return
    days_in_sample = (df['date'].iloc[-1] - df['date'].iloc[0]).days
    annualized_return = ((1 + total_return/100)**(365/days_in_sample) - 1) * 100
    
    # Sharpe ratio (annualized)
    if len(daily_returns) > 1:
        sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(365)
    else:
        sharpe_ratio = 0
    
    # Max drawdown
    cumulative_returns_pct = (cumulative_returns - 1) * 100
    rolling_max = cumulative_returns_pct.cummax()
    drawdown = cumulative_returns_pct - rolling_max
    max_drawdown = drawdown.min()
    
    # Trade metrics
    if not trade_df.empty:
        total_trades = len(trade_df)
        winning_trades = len(trade_df[trade_df['net_return'] > 0])
        win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
        
        avg_win = trade_df[trade_df['net_return'] > 0]['return_pct'].mean() if winning_trades > 0 else 0
        avg_loss = trade_df[trade_df['net_return'] < 0]['return_pct'].mean() if (total_trades - winning_trades) > 0 else 0
        
        gross_profit = trade_df[trade_df['net_return'] > 0]['net_return'].sum()
        gross_loss = abs(trade_df[trade_df['net_return'] < 0]['net_return'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        avg_duration = trade_df['duration_days'].mean()
        expectancy = (win_rate/100 * avg_win) + ((100-win_rate)/100 * avg_loss)
    else:
        total_trades = 0
        win_rate = 0
        avg_win = 0
        avg_loss = 0
        profit_factor = 0
        avg_duration = 0
        expectancy = 0
    
    return {
        'Total Return (%)': total_return,
        'Annualized Return (%)': annualized_return,
        'Annualized Sharpe': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown,
        'Total Trades': total_trades,
        'Win Rate (%)': win_rate,
        'Avg Win (%)': avg_win,
        'Avg Loss (%)': avg_loss,
        'Profit Factor': profit_factor,
        'Avg Trade Duration': avg_duration,
        'Expectancy (%)': expectancy
    }

def run_sensitivity_analysis(df):
    """Run threshold sensitivity analysis"""
    thresholds = np.arange(1, 11, 0.5)  # Test thresholds from 1% to 10%
    results = []
    
    for thresh in thresholds:
        temp_df = backtest_strategy(df, entry_thresh=thresh, exit_thresh=1.0)
        trade_history, trade_df = generate_trade_history(temp_df)
        metrics = calculate_performance_metrics(temp_df, trade_df)
        results.append({
            'Threshold': thresh,
            'Total Return': metrics['Total Return (%)'],
            'Sharpe Ratio': metrics['Annualized Sharpe'],
            'Win Rate': metrics['Win Rate (%)'],
            'Profit Factor': metrics['Profit Factor'],
            'Max Drawdown': metrics['Max Drawdown (%)']
        })
    
    return pd.DataFrame(results)

def generate_trading_report(df, trade_history, trade_df, metrics, sensitivity_df):
    """Generate comprehensive PDF report"""
    report_path = os.path.join(OUTPUT_DIR, 'BTC_Spread_Trading_Report_1.pdf')
    
    with PdfPages(report_path) as pdf:
    
        # Strategy overview
        plt.figure(figsize=(11, 8))
        plt.axis('off')
        
        strategy_text = [
            "Strategy Overview:",
            "",
            "This strategy trades the spread between Bitcoin's annualized basis (futures premium)",
            "& funding rate (Spread = Basis - Funding). The strategy aims to capture mean-reversion",
            "opportunities when the spread deviates significantly from its historical average.",
            "",
            "Position Rules:",
            "  LONG SPREAD: Buy spot + Sell futures when spread > +5%",
            "  SHORT SPREAD: Sell spot + Buy futures when spread < -5%",
            "  EXIT: Close position when spread returns within ±1%",
            "",
            f"Parameters: Entry Threshold ±{ENTRY_THRESHOLD}% | Exit Threshold ±{EXIT_THRESHOLD}%",
            f"Capital per Trade: ${CAPITAL_PER_TRADE:,.0f} | Trading Fees: {TRADING_FEE*100:.2f}% per leg"
        ]
        
        plt.text(0.05, 0.9, '\n'.join(strategy_text), 
                ha='left', va='top', fontsize=11, linespacing=1.5)
        
        # Performance summary table
        plt.text(0.05, 0.4, 'Performance Summary:', 
                ha='left', va='top', fontsize=12, fontweight='bold')
        
        perf_data = [
            ['Total Return', f"{metrics['Total Return (%)']:.2f}%"],
            ['Annualized Return', f"{metrics['Annualized Return (%)']:.2f}%"],
            ['Sharpe Ratio', f"{metrics['Annualized Sharpe']:.2f}"],
            ['Max Drawdown', f"{metrics['Max Drawdown (%)']:.2f}%"],
            ['Total # Roundtrip Trades', metrics['Total Trades']],
            ['Win Rate', f"{metrics['Win Rate (%)']:.1f}%"],
            ['Profit Factor', f"{metrics['Profit Factor']:.2f}"],
            ['Avg Trade Duration', f"{metrics['Avg Trade Duration']:.1f} days"]
        ]
        
        table = plt.table(cellText=perf_data,
                        colWidths=[0.3, 0.2],
                        cellLoc='left',
                        loc='bottom left',
                        bbox=[0.05, 0.1, 0.9, 0.25])
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        
        pdf.savefig()
        plt.close()
        
        # Spread & signals plot
        fig, ax = plt.subplots(2, 1, figsize=(11, 8))
        ax[0].plot(df['date'], df['basis_annual_pct'], label='Annualized Basis', color='blue')  
        ax[0].plot(df['date'], df['funding_annual_pct'], label='Annualized Funding', color='red')  
        ax[0].fill_between(df['date'], df['basis_annual_pct'], df['funding_annual_pct'], 
                        where=(df['basis_annual_pct']>df['funding_annual_pct']),  
                        color='green', alpha=0.3, label='Positive Spread')
        ax[0].axhline(ENTRY_THRESHOLD, color='gray', linestyle='--', alpha=0.5)
        ax[0].axhline(-ENTRY_THRESHOLD, color='gray', linestyle='--', alpha=0.5)
        ax[0].set_title('BTC Basis vs Funding Rate Spread')
        ax[0].legend()
        ax[0].set_ylabel('Annualized %')

        ax[1].plot(df['date'], df['position'], label='Strategy Position', 
                drawstyle='steps-post', color='purple', linewidth=2)
        ax[1].axhline(0, color='black', linestyle='-', alpha=0.5)
        ax[1].set_yticks([-1, 0, 1])
        ax[1].set_yticklabels(['Short\nSpread', 'Flat', 'Long\nSpread'])
        ax[1].set_title('Strategy Signals')
        plt.tight_layout()
        pdf.savefig()
        plt.close()
        
        # Drawdown
        plt.figure(figsize=(11, 4))
        equity = (1 + df['daily_returns'].fillna(0)).cumprod()
        rolling_max = equity.cummax()
        drawdown = (equity - rolling_max) / rolling_max * 100
        
        drawdown.plot(color='red', label='Drawdown')
        plt.fill_between(drawdown.index, drawdown, color='red', alpha=0.3)
        plt.title('Strategy Drawdown')
        plt.ylabel('Drawdown (%)')
        plt.legend()
        pdf.savefig()
        plt.close()
        
        # Trade analysis
        if not trade_df.empty:
            # Returns distribution
            plt.figure(figsize=(11, 6))
            plt.subplot(121)
            sns.histplot(trade_df['return_pct'], bins=20, kde=True)
            plt.title('Trade Returns Distribution')
            plt.xlabel('Return (%)')
            
            # Duration vs Returns
            plt.subplot(122)
            sns.scatterplot(data=trade_df, x='duration_days', y='return_pct', hue='direction')
            plt.title('Trade Duration vs Returns')
            plt.xlabel('Duration (days)')
            plt.ylabel('Return (%)')
            plt.tight_layout()
            pdf.savefig()
            plt.close()
            
            # Cum. returns by trade
            plt.figure(figsize=(11, 6))
            trade_df = trade_df.sort_values('exit_date')
            trade_df['cumulative_return'] = (1 + trade_df['return_pct']/100).cumprod() * CAPITAL_PER_TRADE
            plt.plot(trade_df['exit_date'], trade_df['cumulative_return'])
            plt.title('Cumulative Returns by Trade')
            plt.ylabel('Portfolio Value ($)')
            pdf.savefig()
            plt.close()
        
        # Sensitivity analysis
        plt.figure(figsize=(11, 8))
        plt.subplot(221)
        plt.plot(sensitivity_df['Threshold'], sensitivity_df['Total Return'], 'o-')
        plt.title('Total Return by Entry Threshold')
        plt.xlabel('Threshold (%)')
        plt.ylabel('Return (%)')
        
        plt.subplot(222)
        plt.plot(sensitivity_df['Threshold'], sensitivity_df['Sharpe Ratio'], 'o-')
        plt.title('Sharpe Ratio by Entry Threshold')
        plt.xlabel('Threshold (%)')
        plt.ylabel('Sharpe Ratio')
        
        plt.subplot(223)
        plt.plot(sensitivity_df['Threshold'], sensitivity_df['Win Rate'], 'o-')
        plt.title('Win Rate by Entry Threshold')
        plt.xlabel('Threshold (%)')
        plt.ylabel('Win Rate (%)')
        
        plt.subplot(224)
        plt.plot(sensitivity_df['Threshold'], sensitivity_df['Max Drawdown'], 'o-')
        plt.title('Max Drawdown by Entry Threshold')
        plt.xlabel('Threshold (%)')
        plt.ylabel('Drawdown (%)')
        
        plt.tight_layout()
        pdf.savefig()
        plt.close()
        
        # Trade history
        if not trade_history.empty:
            plt.figure(figsize=(11, 8))
            ax = plt.subplot(111)
            ax.axis('off')
            
            display_cols = ['date', 'type', 'capital', 'spot_price', 'futures_price', 'spread', 'duration_days', 'return_pct']
            trades_display = trade_history[display_cols].copy()
            
            trades_display['date'] = trades_display['date'].dt.strftime('%Y-%m-%d')
            trades_display['capital'] = trades_display['capital'].apply(lambda x: f"${x:,.2f}")
            trades_display['spot_price'] = trades_display['spot_price'].apply(lambda x: f"${x:,.2f}")
            trades_display['futures_price'] = trades_display['futures_price'].apply(lambda x: f"${x:,.2f}")
            trades_display['spread'] = trades_display['spread'].apply(lambda x: f"{x:.2f}%")
            trades_display['return_pct'] = trades_display['return_pct'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
            
            table = ax.table(cellText=trades_display.values,
                            colLabels=trades_display.columns,
                            cellLoc='center',
                            loc='center')
            
            table.auto_set_font_size(False)
            table.set_fontsize(9)
            table.scale(1, 1.2)
            
            for (row, col), cell in table.get_celld().items():
                if row == 0:
                    cell.set_text_props(weight='bold', color='white')
                    cell.set_facecolor('#4b61d1')
                else:
                    if 'enter' in trades_display.iloc[row-1]['type']:
                        cell.set_facecolor('#f0f7f0')
                    elif 'exit' in trades_display.iloc[row-1]['type']:
                        try:
                            return_pct = float(trades_display.iloc[row-1]['return_pct'].replace('%', ''))
                            cell.set_facecolor('#e6f3e6' if return_pct > 0 else '#f3e6e6')
                        except:
                            cell.set_facecolor('#f5f5f5')
            
            plt.title('Trade History', pad=20, fontsize=14, fontweight='bold')
            pdf.savefig()
            plt.close()

# Main execution
if __name__ == "__main__":
    try:
        print("Starting BTC Spread Trading Analysis...")
        
        print("Loading and cleaning price data...")      
        df = load_clean_data()
        
        print("Running backtest...")
        results = backtest_strategy(df)
    
        print("Analyzing trade history...")
        trade_history, trade_df = generate_trade_history(results)
        
        print("Calc performance metrics...")
        metrics = calculate_performance_metrics(results, trade_df)
        
        print("Running sensitivity analysis...")
        sensitivity_df = run_sensitivity_analysis(df)
        
        print("Generating PDF...")
        generate_trading_report(results, trade_history, trade_df, metrics, sensitivity_df)
        
    
        trade_history.to_csv(
            os.path.join(INPUT_DIR, 'btc_trade_history.csv'),
            index=False,
            float_format='%.6f',
            quoting=csv.QUOTE_NONNUMERIC,
            quotechar='"',
            encoding='utf-8')
        
        results.to_csv(
            os.path.join(INPUT_DIR, 'btc_backtest_results.csv'),
            index=False,
            float_format='%.6f',
            quoting=csv.QUOTE_MINIMAL,
            quotechar='"',
            encoding='utf-8',
            date_format='%Y-%m-%d')
        
        print(f"\nAnalysis completed successfully!")
        # [Rest of your print statements]
        
    except Exception as e:
        print(f"\nError in analysis: {str(e)}")
  
        print(f"\nAnalysis completed successfully!")
        print(f"Report generated: {os.path.join(OUTPUT_DIR, 'BTC_Spread_Trading_Report_1.pdf')}")
        print("\nKey Performance Metrics:")
        print(f"Total Return: {metrics['Total Return (%)']:.2f}%")
        print(f"Annualized Return: {metrics['Annualized Return (%)']:.2f}%")
        print(f"Sharpe Ratio: {metrics['Annualized Sharpe']:.2f}")
        print(f"Win Rate: {metrics['Win Rate (%)']:.1f}%")
        print(f"Profit Factor: {metrics['Profit Factor']:.2f}")
        print(f"Max Drawdown: {metrics['Max Drawdown (%)']:.2f}%")
        
    except Exception as e:
        print(f"\nError in analysis: {str(e)}")    
                 

In [ ]:
import os
import sys
from importlib.util import find_spec

def ensure_pypdf2():
    """Ensure PyPDF2 is available with fallbacks"""
    if find_spec("PyPDF2"):
        try:
            from PyPDF2 import PdfMerger
            return PdfMerger
        except ImportError:
            try:
                from PyPDF2 import PdfFileMerger as PdfMerger
                return PdfMerger
            except ImportError:
                pass
    
    # If we get here, installation is needed
    print("Installing PyPDF2...")
    try:
        import pip
        pip.main(['install', 'PyPDF2'])
    except:
        import subprocess
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'PyPDF2'])
    
    # Try imports again after installation
    from PyPDF2 import PdfMerger
    return PdfMerger

def combine_pdfs(pdf1_path, pdf2_path, output_path):
    """Robust PDF combining with installation fallback"""
    PdfMerger = ensure_pypdf2()
    input_files = [pdf1_path, pdf2_path]  # Store input files for potential deletion
    
    try:
        # Convert to absolute paths
        pdf1_path = os.path.abspath(pdf1_path)
        pdf2_path = os.path.abspath(pdf2_path)
        
        print(f"Looking for files at:\n- {pdf1_path}\n- {pdf2_path}")
        
        merger = PdfMerger()
        for pdf in input_files:
            if not os.path.exists(pdf):
                raise FileNotFoundError(f"PDF file not found: {pdf}")
            merger.append(pdf)
        
        # Create output directory if needed
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        merger.write(output_path)
        merger.close()
        print(f"Successfully created: {os.path.abspath(output_path)}")
        
        # Delete input files after successful combination
        for pdf in input_files:
            try:
                os.remove(pdf)
                print(f"Deleted input file: {pdf}")
            except Exception as e:
                print(f"Warning: Could not delete {pdf} - {str(e)}")
        
        return True
    except Exception as e:
        print(f"Error combining PDFs: {str(e)}")
        return False

# Example usage with verification
if __name__ == "__main__":
    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")
    
    # Define the reports subdirectory
    reports_dir = os.path.join(cwd, "reports")
    
    # List files in reports directory to help debugging
    print("\nFiles in reports directory:")
    try:
        for f in os.listdir(reports_dir):
            if f.lower().endswith('.pdf'):
                print(f"- {f}")
    except FileNotFoundError:
        print(f"Reports directory not found at: {reports_dir}")
    
    # Try combining with correct paths
    input_pdf1 = os.path.join(reports_dir, "BTC_Spread_Trading_Report_0.pdf")
    input_pdf2 = os.path.join(reports_dir, "BTC_Spread_Trading_Report_1.pdf")
    output_pdf = os.path.join(reports_dir, "BTC_Spread_Trading_Report_Final.pdf")
    
    print(f"\nAttempting to combine:\n- {input_pdf1}\n- {input_pdf2}")
    
    if combine_pdfs(input_pdf1, input_pdf2, output_pdf):
        print("Combination successful!")
        print(f"Output file created at: {output_pdf}")
    else:
        print("Failed to combine PDFs")
        print("\nPossible solutions:")
        print("1. Verify the PDF files exist in the reports subdirectory")
        print("2. Check the filenames are correct (case-sensitive)")
        print("3. Ensure you have read/write permissions for the reports directory")